# General Instructions to students:

1. There are 5 types of cells in this notebook. The cell type will be indicated within the cell.
    1. Markdown cells with problem written in it. (DO NOT TOUCH THESE CELLS) (**Cell type: TextRead**)
    2. Python cells with setup code for further evaluations. (DO NOT TOUCH THESE CELLS) (**Cell type: CodeRead**)
    3. Python code cells with some template code or empty cell. (FILL CODE IN THESE CELLS BASED ON INSTRUCTIONS IN CURRENT AND PREVIOUS CELLS) (**Cell type: CodeWrite**)
    4. Markdown cells where a written reasoning or conclusion is expected. (WRITE SENTENCES IN THESE CELLS) (**Cell type: TextWrite**)
    5. Temporary code cells for convenience and TAs. (YOU MAY DO WHAT YOU WILL WITH THESE CELLS, TAs WILL REPLACE WHATEVER YOU WRITE HERE WITH OFFICIAL EVALUATION CODE) (**Cell type: Convenience**)
    
2. You are not allowed to insert new cells in the submitted notebook.

3. You are not allowed to import any extra packages.

4. The code is to be written in Python 3.6 syntax. Latest versions of other packages maybe assumed.

5. In CodeWrite Cells, the only outputs to be given are plots asked in the question. Nothing else to be output/print. 

6. If TextWrite cells ask you to give accuracy/error/other numbers you can print them on the code cells, but remove the print statements before submitting.

7. The convenience code can be used to check the expected syntax of the functions. At a minimum, your entire notebook must run with "run all" with the convenience cells as it is. Any runtime failures on the submitted notebook as it is will get zero marks.

8. All code must be written by yourself. Copying from other students/material on the web is strictly prohibited. Any violations will result in zero marks.

9. All datasets will be given as .npz files, and will contain data in 4 numpy arrays :"X_train, Y_train, X_test, Y_test". In that order. The meaning of the 4 arrays can be easily inferred from their names.

10. All plots must be labelled properly, all tables must have rows and columns named properly.

11. Before subbmission ensure that you submit with the outputs (do not clear the outputs), so that when evaluating we can run selectively.

12. Before submission ensure that the path for the folder containing the data is "../../Data/" 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import matplotlib as mpl
import pandas as pd

# 1. Logistic Regression 

Write code for doing logistic regression below. Also write code for choosing best hyperparameters for each kernel type (use a part of training set as validation set). 

The range of hyperparameters is typically chosen on a log scale e.g. 1e-4, 1e-3, 1e-2... 1e3.

Write code for running in the cell after (You may be asked to demonstrate your code during the viva using this cell.)

In text cell after that report the following numbers you get by running appropriate code:

For each classification data set report the best kernel and regularisation parameters for linear, RBF and Poly kernels. (Linear has no kernel parameter.) Report the training and test zero-one error for those hyperparameters. 

For each given hyperparameter setting (kernel and regularisation) you will have to do some exploring to find the right learning rate to use in gradient descent. The optimisation learning rate is not a model hyperparameter and hence can be chosen based on just the training set. i.e. choose the learning rate for which the training loss decreases the most.

For the synthetic classification datasets (dataset_A and dataset_B) in 2-dimensions, also illustrate the learned classifier for each kernel setting. Do this in the last codeWrite cell for this question.


In [5]:
# CodeWrite 
# Write logistic regression code from scratch. Use gradient descent.
# Only write functions here

def calculate_kernel_matrix(X, kernel, kernel_param):
    """
    Calculate the kernel matrix for the data X
    """
    if(kernel == 'linear'):
        return np.matmul(X, X.T)
    elif(kernel == 'poly'):
        return (1 + np.matmul(X, X.T)) ** kernel_param
    elif(kernel == 'rbf'):
        # kernel(X, Y) = exp(-gamma * ||X - Y||^2)
        return (np.exp(-kernel_param * (np.sum(X**2, axis=1)[:, np.newaxis] + np.sum(X**2, axis=1) - 2 * np.matmul(X, X.T))))

def calculate_kernel_values(X, x, kernel, kernel_param):
    """
    Calculate the kernel values between x and all the points in X
    """
    if kernel == 'linear':
        return np.dot(X, x)
    elif kernel == 'poly':
        return (1 + np.dot(X, x)) ** kernel_param
    elif kernel == 'rbf':
        return np.exp(-kernel_param * np.sum((X - x) ** 2, axis=1))

def train_pred_logistic_regression(X, Y, kernel='linear', reg_param=0., kernel_param=1., num_iter_gd=100):
    """
    Arguments:
    X : (n,d) shape numpy array
    Y : (n,)  shape numpy array
    X_test : (m,d) shape numpy array
    kernel = 'linear' or 'rbf' or 'poly' 
    reg_param = $\ lambda$
    num_iter_gd = number of GD iterations.

    Returns the result of kernel logistic regression :
    alpha: Vector of solutions for the dual. Numpy array of shape (n,)

    Primal problem:
    $ \ min_w  \ sum_{i=1}^n \ log(1+\ exp(-y_i* \ w^\ top \phi(\ x_i)))  + \ frac{\ lambda}{2} ||\ w||^2 $

    the dual of which is

    $ \ min_alpha \ sum_{i=1}^n \ log(1+\ exp(-y_i* \ alpha^\ top K_{:,i} ))  + \ frac{\ lambda}{2} \ alpha^\ top K \ alpha $
    where $\ phi$ is the feature got by the kernel.

    Where K is the nxn kernel matrix computed on the training data.

    The kernel is defined by the kernel_param:
    If kernel=linear: K(\ u,\ v) = \ u^\ top \ v  
    If kernel=poly:  K(\ u,\ v) = (1+\ u^\ top \ v)^(kernel_param)
    If kernel=rbf:  K(\ u,\ v) = \ exp(-kernel_param*||\ u-\ v||^2)
    """
    kernel_X = calculate_kernel_matrix(X, kernel, kernel_param)
    alpha = np.zeros(X.shape[0])
    eta = 0.1
    
    for i in range(num_iter_gd):
        delta_R_alpha = reg_param * alpha - np.sum((1 - 1 / (1 + np.exp(-Y * np.matmul(kernel_X, alpha)))) * Y * kernel_X, axis=1)
        alpha = alpha - eta * delta_R_alpha
    
    return alpha

def test_pred(alpha, train_X, train_Y, test_X, kernel, kernel_param):
    """
    Return the predictions on test_X using the learnt alphas
    """
    return np.array([np.sign(np.dot(alpha, calculate_kernel_values(train_X, test_X[i], kernel, kernel_param))) for i in range(test_X.shape[0])])

In [6]:
# CodeWrite : Use the functions above to do validation to get best hyperparameters 
# (i.e. kernel_param and regularisation_param).
# Also, get the numbers you report below.

import warnings
warnings.filterwarnings('ignore')

kernels = ['linear', 'poly', 'rbf']
num_iter = 100

best_alphas_A = {}
best_alphas_B = {}
best_kernel_param_A = {}
best_kernel_param_B = {}
best_reg_param_A = {}
best_reg_param_B = {}

def find_best_parameter(kernel, X_train, Y_train, X_val, Y_val):
    best_kernel_param, best_reg_param, best_accuracy = 0, 0, 0

    if kernel == 'linear':
        kernel_params = [1]
        reg_params = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
    elif kernel == 'poly':
        kernel_params = [1, 2, 4, 6, 8, 10]
        reg_params = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
    elif kernel == 'rbf':
        kernel_params = [1e-1, 1, 2, 4, 10, 100]
        reg_params = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]

    for kernel_param in kernel_params:
        for reg_param in reg_params:
            alpha = train_pred_logistic_regression(X_train, Y_train, kernel=kernel, reg_param=reg_param, kernel_param=kernel_param, num_iter_gd=num_iter)
            pred = test_pred(alpha, X_train, Y_train, X_val, kernel=kernel, kernel_param=kernel_param)
            accuracy = np.mean(pred == Y_val)
            # print("Kernel: {}, kernel_param: {}, reg_param: {}, val_accuracy: {}".format(kernel, kernel_param, reg_param, accuracy))
            if accuracy > best_accuracy:
                best_kernel_param, best_reg_param, best_accuracy = kernel_param, reg_param, accuracy
    return best_kernel_param, best_reg_param, best_accuracy

for a in ['A', 'B', 'C', 'D']:
    data = np.load('../../Data/dataset_{}.npz'.format(a))
    X_train, Y_train = data['arr_0'], data['arr_1']
    X_test, Y_test = data['arr_2'], data['arr_3']

    print("Dataset {}\n".format(a))
    X_train, X_val = X_train[:int(0.8 * X_train.shape[0])], X_train[int(0.8 * X_train.shape[0]):]
    Y_train, Y_val = Y_train[:int(0.8 * Y_train.shape[0])], Y_train[int(0.8 * Y_train.shape[0]):]

    for kernel in kernels:
        best_kernel_param, best_reg_param, best_accuracy = find_best_parameter(kernel, X_train, Y_train, X_val, Y_val)
        print("Kernel: {}, best_kernel_param: {}, best_reg_param: {}, best_val_accuracy: {}".format(kernel, best_kernel_param, best_reg_param, best_accuracy))

        alpha = train_pred_logistic_regression(X_train, Y_train, kernel, best_reg_param, best_kernel_param)

        if(a == 'A'):
            best_alphas_A[kernel], best_kernel_param_A[kernel], best_reg_param_A[kernel] = alpha, best_kernel_param, best_reg_param
        elif(a == 'B'):
            best_alphas_B[kernel], best_kernel_param_B[kernel], best_reg_param_B[kernel] = alpha, best_kernel_param, best_reg_param

        pred = test_pred(alpha, X_train, Y_train, X_test, kernel, best_kernel_param)
        accuracy = np.mean(pred == Y_test)
        print("Test accuracy: {}\n".format(accuracy))

Dataset A

Kernel: linear, best_kernel_param: 1, best_reg_param: 1e-05, best_val_accuracy: 0.8266666666666667
Test accuracy: 0.806

Kernel: poly, best_kernel_param: 2, best_reg_param: 0.0001, best_val_accuracy: 0.87
Test accuracy: 0.86

Kernel: rbf, best_kernel_param: 1, best_reg_param: 1e-06, best_val_accuracy: 1.0
Test accuracy: 0.994

Dataset B

Kernel: linear, best_kernel_param: 1, best_reg_param: 1e-06, best_val_accuracy: 0.74
Test accuracy: 0.758

Kernel: poly, best_kernel_param: 4, best_reg_param: 1e-05, best_val_accuracy: 0.8333333333333334
Test accuracy: 0.776

Kernel: rbf, best_kernel_param: 2, best_reg_param: 0.0001, best_val_accuracy: 0.8333333333333334
Test accuracy: 0.784

Dataset C

Kernel: linear, best_kernel_param: 1, best_reg_param: 1e-06, best_val_accuracy: 0.5333333333333333
Test accuracy: 0.5016835016835017

Kernel: poly, best_kernel_param: 8, best_reg_param: 0.01, best_val_accuracy: 0.96
Test accuracy: 0.9494949494949495

Kernel: rbf, best_kernel_param: 0.1, best_

TextWrite Cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error for all three kernel choices, for all 4 datasets (2 real world and 2 synthetic).  




In [ ]:
# Codewrite cell: Generate plots of learned classifier for all three kernel types, on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 3 * 2 = 6

kernels = ['linear', 'poly', 'rbf']

def plot(alpha, X_test, Y_test, kernel, kernel_param, reg_param, dataset):
    minX, maxX = np.min(X_test[:, 0]), np.max(X_test[:, 0])
    minY, maxY = np.min(X_test[:, 1]), np.max(X_test[:, 1])
    points = 500
    X, Y = np.meshgrid(np.linspace(minX - 0.2, maxX + 0.2, points), np.linspace(minY - 0.2, maxY + 0.2, points))
    Z = np.zeros((points, points))
    X_plotter = np.zeros((points * points, 2))
    for i in range(points):
        for j in range(points):
            X_plotter[i * points + j, 0] = X[i, j]
            X_plotter[i * points + j, 1] = Y[i, j]
    pred = test_pred(alpha, X_train, Y_train, X_plotter, kernel=kernel, kernel_param=kernel_param)
    Z = pred.reshape((points, points))
    plt.contourf(X, Y, Z, alpha=0.2, colors=['r', 'g'])
    plt.title("{} - Kernel: {}, KP: {}, RP: {}".format(dataset, kernel, kernel_param, reg_param))
    plt.scatter(X_test[Y_test == 1, 0], X_test[Y_test == 1, 1], marker='.', color='g')
    plt.scatter(X_test[Y_test == -1, 0], X_test[Y_test == -1, 1], marker='.', color='r')
    plt.show()

for a in ['A', 'B']:
    data = np.load('../../Data/dataset_{}.npz'.format(a))
    X_train = data['arr_0']
    Y_train = data['arr_1']
    X_test = data['arr_2']
    Y_test = data['arr_3']

    X_train = X_train[:int(X_train.shape[0] * 0.8)]
    Y_train = Y_train[:int(Y_train.shape[0] * 0.8)]
    
    print("Dataset {}".format(a))
    for kernel in kernels:
        if(a == 'A'):
            alpha = best_alphas_A[kernel]
            kernel_param = best_kernel_param_A[kernel]
            reg_param = best_reg_param_A[kernel]
        else:
            alpha = best_alphas_B[kernel]
            kernel_param = best_kernel_param_B[kernel]
            reg_param = best_reg_param_B[kernel]
        plot(alpha, X_test, Y_test, kernel, kernel_param, reg_param, "Dataset {}".format(a))

# 2. SVM

Write code for learning SVM below. Also write code for choosing best hyperparameters for each kernel type. You may use sklearn.svm for this purpose. (use a part of training set as validation set)

Write code for running in the cell after (You may be asked to demonstrate your code during the viva using this cell.)

In text cell after that report the following numbers you get by running appropriate code:

For each classification data set report the best kernel and regularisation parameters for linear, RBF and Poly kernels. (Linear has no kernel parameter.) Report the training and test zero-one error for those hyperparameters.

For the synthetic classification datasets in 2-dimensions, also illustrate the learned classifier for each kernel setting. Do this in the last codeWrite cell for this question.

In [ ]:
# CodeWrite cell
# Write SVM classifier using SKlearn
# write only functions here

def train_svm(X, Y, kernel='rbf', reg_param=1.0, kernel_param=1.0):
    """
    Train SVM classifier using sklearn.svm.SVC
    """
    clf = svm.SVC(kernel=kernel, C=reg_param)
    if kernel == 'rbf':
        clf.set_params(**{'gamma': kernel_param})
    elif kernel == 'poly':
        clf.set_params(**{'degree': kernel_param, 'coef0': 1.0})
    clf.fit(X, Y)
    return clf

def predict_value(clf, X):
    """
    Predict using the SVM classifier
    """
    return clf.predict(X)

In [ ]:
# CodeWrite cell
# Write code here for doing validation (for kernel_param and regularisation_param)
# on a subset of the training set. 
# Also for generating the numbers that you report below.

def normalize(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

kernels = ['linear', 'poly', 'rbf']

best_clf_A = {}
best_clf_B = {}
best_kernel_param_A = {}
best_reg_param_A = {}
best_kernel_param_B = {}
best_reg_param_B = {}

def find_best_parameter(kernel, X_train, Y_train, X_val, Y_val):
    best_kernel_param, best_reg_param, best_accuracy = None, None, 0
    
    if kernel == 'linear':
        kernel_params = [1]
        reg_params = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]
    elif kernel == 'poly':
        kernel_params = [1, 2, 4, 6, 8]
        reg_params = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]
    elif kernel == 'rbf':
        kernel_params = [1e-3, 1e-2, 1e-1, 1, 2, 4, 10, 100]
        reg_params = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

    for kernel_param in kernel_params:
        for reg_param in reg_params:
            clf = train_svm(X_train, Y_train, kernel, reg_param=reg_param, kernel_param=kernel_param)
            pred = predict_value(clf, X_val)
            accuracy = np.mean(pred == Y_val)
            print("kernel: {}, kernel_param: {}, reg_param: {}, accuracy: {}".format(kernel, kernel_param, reg_param, accuracy))
            if accuracy > best_accuracy:
                best_kernel_param, best_reg_param, best_accuracy = kernel_param, reg_param, accuracy
    return best_kernel_param, best_reg_param, best_accuracy

for a in ['A', 'B', 'C', 'D']:
    dataset = "../../Data/dataset_" + a + ".npz"
    data = np.load(dataset)

    print("Dataset {}\n".format(a))
    X_train, Y_train = data['arr_0'], data['arr_1']
    X_test, Y_test = data['arr_2'], data['arr_3']

    rand_indices = np.random.permutation(X_train.shape[0])
    X_train, X_val = X_train[rand_indices[:int(X_train.shape[0] * 0.8)]], X_train[rand_indices[int(X_train.shape[0] * 0.8):]]
    Y_train, Y_val = Y_train[rand_indices[:int(Y_train.shape[0] * 0.8)]], Y_train[rand_indices[int(Y_train.shape[0] * 0.8):]]

    for kernel in kernels:
        best_kernel_param, best_reg_param, best_accuracy = find_best_parameter(kernel, X_train, Y_train, X_val, Y_val)
        print("Kernel: {}, best_kernel_param: {}, best_reg_param: {}, val_accuracy: {}".format(kernel, best_kernel_param, best_reg_param, best_accuracy))
        clf = train_svm(X_train, Y_train, kernel=kernel, reg_param=best_reg_param, kernel_param=best_kernel_param)
        if(a == 'A'):
            best_clf_A[kernel], best_kernel_param_A[kernel], best_reg_param_A[kernel] = clf, best_kernel_param, best_reg_param
        elif(a == 'B'):
            best_clf_B[kernel], best_kernel_param_B[kernel], best_reg_param_B[kernel] = clf, best_kernel_param, best_reg_param
                          
        pred = predict_value(clf, X_test)
        accuracy = np.mean(pred == Y_test)
        print("Test accuracy: {}\n".format(accuracy))

TextWrite Cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error for all three kernel choices, for all 4 datasets (2 real world and 2 synthetic).  


In [ ]:
# Codewrite cell: Generate plots of learned classifier for all three kernel types, on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 3 * 2 = 6

def plot(clf, X_test, Y_test, dataset, kernel, kernel_param, reg_param):
    minX, maxX = np.min(X_test[:, 0]), np.max(X_test[:, 0])
    minY, maxY = np.min(X_test[:, 1]), np.max(X_test[:, 1])
    points = 500
    X, Y = np.meshgrid(np.linspace(minX - 0.2, maxX + 0.2, points), np.linspace(minY - 0.2, maxY + 0.2, points))
    Z = np.zeros((points, points))
    for i in range(points):
        for j in range(points):
            Z[i][j] = clf.predict(np.array([[X[i][j], Y[i][j]]]))
    plt.contourf(X, Y, Z, alpha=0.2, colors=['r', 'g'])
    plt.title("{}, Kernel: {}, KP: {}, RP: {}".format(dataset, kernel, kernel_param, reg_param))
    plt.scatter(X_test[Y_test == 1, 0], X_test[Y_test == 1, 1], marker='.', color='g')
    plt.scatter(X_test[Y_test == -1, 0], X_test[Y_test == -1, 1], marker='.', color='r')
    plt.show()

for a in ['A', 'B']:
    data = np.load('../../Data/dataset_{}.npz'.format(a))
    X_train = data['arr_0']
    Y_train = data['arr_1']
    X_test = data['arr_2']
    Y_test = data['arr_3']

    X_train = X_train[:int(X_train.shape[0] * 0.8)]
    Y_train = Y_train[:int(Y_train.shape[0] * 0.8)]
    
    print("Dataset {}".format(a))
    for kernel in kernels:
        if(a == 'A'):
            clf, kernel_param, reg_param = best_clf_A[kernel], best_kernel_param_A[kernel], best_reg_param_A[kernel]
        elif(a == 'B'):
            clf, kernel_param, reg_param = best_clf_B[kernel], best_kernel_param_B[kernel], best_reg_param_B[kernel]
        plot(clf, X_test, Y_test, "Dataset {}".format(a), kernel, kernel_param, reg_param)

# 3. Decision Tree

Write code for learning decision tree below. Take as an argument a hyperparameter on what size node to stop splitting. Use a part of training set as validation set.

Write code for running in the cell after (You may be asked to demonstrate your code during the viva using this cell.)

In text cell after that report the following numbers you get by running appropriate code:

For all four data sets  report the best node size to stop splitting. Report the training and test zero-one error for those hyperparameters.

For datasets A and B, also illustrate the learned classifier. Do this in the last codeWrite cell for this question.

Important: Think about how you will represent a decision tree. (Possible soln: Store as a list of tuples containing node position, attribute to split, threshold, class to classifiy (if leaf node) )


In [24]:
# CodeWrite cell
# Write Decision tree classifier from scratch, 
# write only functions here (you may write extra functions here if you wish)

def find_entropy(X_left_count, X_right_count, X_left_pos, X_right_pos):
    if(X_left_count == 0 or X_right_count == 0):
        return 0
    if(X_left_pos == 0 or X_right_pos == 0):
        return 0
    X_left_neg = X_left_count - X_left_pos
    X_right_neg = X_right_count - X_right_pos
    entropy_left, entropy_right = 0, 0
    entropy_left += -1 * (X_left_pos / X_left_count) * np.log2(X_left_pos / X_left_count)
    entropy_left += -1 * (X_left_neg / X_left_count) * np.log2(X_left_neg / X_left_count)
    entropy_right += -1 * (X_right_pos / X_right_count) * np.log2(X_right_pos / X_right_count)
    entropy_right += -1 * (X_right_neg / X_right_count) * np.log2(X_right_neg / X_right_count)
    return entropy_left * (X_left_count / (X_left_count + X_right_count)) + entropy_right * (X_right_count / (X_left_count + X_right_count))

def find_accuracy(X_left_count, X_right_count, X_left_pos, X_right_pos):
    X_left_neg = (X_left_count - X_left_pos)
    X_right_neg = (X_right_count - X_right_pos)
    total = X_left_count + X_right_count
    return max((X_left_pos + X_right_neg) / total, (X_left_neg + X_right_pos) / total, (X_left_pos + X_right_pos) / total, (X_left_neg + X_right_neg) / total)

def find_best_attribute_threshold(X, Y, criterion):
    best_attribute, best_threshold = -1, -1
    best_accuracy, best_entropy = 0, np.inf

    for attribute in range(X.shape[1]):
        sorted_thresholds = np.sort(np.unique(X[:, attribute]))
        sorted_X, sorted_Y = np.sort(X[:, attribute]), Y[np.argsort(X[:, attribute])]
        X_left_count, X_right_count = 0, X.shape[0]
        X_left_pos, X_right_pos = 0, np.sum(sorted_Y == 1)
        for threshold in sorted_thresholds:
            while(X_left_count < X.shape[0] and sorted_X[X_left_count] < threshold):
                X_left_count += 1
                X_right_count -= 1
                if(X_left_count < X.shape[0] and sorted_Y[X_left_count-1] == 1):
                    X_left_pos += 1
                    X_right_pos -= 1
            if(criterion == 'entropy'):
                entropy = find_entropy(X_left_count, X_right_count, X_left_pos, X_right_pos)
                if(entropy < best_entropy):
                    best_entropy, best_attribute, best_threshold = entropy, attribute, threshold
            elif(criterion == 'accuracy'):
                accuracy = find_accuracy(X_left_count, X_right_count, X_left_pos, X_right_pos)
                if(accuracy > best_accuracy):
                    best_accuracy, best_attribute, best_threshold = accuracy, attribute, threshold
    return best_attribute, best_threshold

class Node:
    def __init__(self, attribute, type, value, threshold):
        self.attribute = attribute
        self.type = type # 0: leaf, 1: internal
        self.value = value # if leaf, value is the class label, else None
        self.threshold = threshold # if internal, threshold is the threshold, else None
        self.left = None
        self.right = None

    def split_node(self, X, Y, criterion, num_nodes_stop):
        if(X.shape[0] <= num_nodes_stop):
            self.type = 0
            self.value = np.sign(np.sum(Y))
            self.value = 1 if self.value == 0 else self.value
            return self

        self.type = 1
        self.attribute, self.threshold = find_best_attribute_threshold(X, Y, criterion)
        X_left, X_right = X[X[:, self.attribute] < self.threshold], X[X[:, self.attribute] >= self.threshold]
        Y_left, Y_right = Y[X[:, self.attribute] < self.threshold], Y[X[:, self.attribute] >= self.threshold]

        if(X_left.shape == X.shape or X_right.shape == X.shape):
            self.type = 0
            self.value = np.sign(np.sum(Y))
            self.value = 1 if self.value == 0 else self.value
            return self

        node_left, node_right = Node(None, None, None, None), Node(None, None, None, None)
        self.left = node_left.split_node(X_left, Y_left, criterion, num_nodes_stop)
        self.right = node_right.split_node(X_right, Y_right, criterion, num_nodes_stop)
        return self

    def predict(self, X):
        if(self.type == 0):
            return self.value
        else:
            if(X[self.attribute] < self.threshold):
                return self.left.predict(X)
            else:
                return self.right.predict(X)

def train_decision_tree(X, Y, num_nodes_stop=1, criterion='accuracy'):
    """ 
    Returns a decision tree trained on X and Y. 
    Stops splitting nodes when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Returns a tree (In whatever format that you find appropriate)
    """
    root = Node(None, 0, None, None)
    root.split_node(X, Y, criterion, num_nodes_stop)
    return root  

def eval_decision_tree(tree, test_X):
    """ 
    Takes in a tree, and a bunch of instances X and 
    returns the tree predicted values at those instances.
    """
    return np.array([tree.predict(x) for x in test_X])

In [25]:
# CodeWrite cell
# Write code here for doing validation to find the best hyperparameters (i.e. num_nodes_stop)
# Also Generate the numbers that you report below. 
# Repeat with criterion set to entropy also.

def plot(tree, X_test, Y_test, criterion, num_nodes_stop, dataset):
    minX, maxX = np.min(X_test[:, 0]), np.max(X_test[:, 0])
    minY, maxY = np.min(X_test[:, 1]), np.max(X_test[:, 1])

    X, Y = np.meshgrid(np.linspace(minX - 0.2, maxX + 0.2, 300), np.linspace(minY - 0.2, maxY + 0.2, 300))
    Z = np.array([tree.predict(np.array([x, y])) for x, y in zip(np.ravel(X), np.ravel(Y))])
    Z = Z.reshape(X.shape)
    plt.contourf(X, Y, Z, alpha=0.2, colors=['red', 'blue'])
    plt.title('{} with criterion = {} and num_nodes_stop = {}'.format(dataset, criterion, num_nodes_stop))
    plt.scatter(X_test[Y_test == 1, 0], X_test[Y_test == 1, 1], color='blue', marker='.', label='Class 1')
    plt.scatter(X_test[Y_test == -1, 0], X_test[Y_test == -1, 1], color='red', marker='.', label='Class -1')
    plt.show() 

for a in ['A', 'B', 'C', 'D']:
    data = np.load("../../Data/dataset_{}.npz".format(a))
    X_train, Y_train = data['arr_0'], data['arr_1']
    X_test, Y_test = data['arr_2'], data['arr_3']
    print('Dataset {}'.format(a))

    num_nodes_stops = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    criterions = ['accuracy', 'entropy']
    for num_nodes_stop in num_nodes_stops:
        for criterion in criterions:
            tree = train_decision_tree(X_train, Y_train, num_nodes_stop, criterion)
            Y_pred = eval_decision_tree(tree, X_test)
            accuracy = np.mean(Y_pred == Y_test)
            print("Accuracy: {}, Criterion: {}, Num_nodes_stop: {}".format(accuracy, criterion, num_nodes_stop))
            # if a == 'A' or a == 'B':
            #     plot(tree, X_test, Y_test, criterion, num_nodes_stop, "Dataset {}".format(a))


Dataset A
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 1
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 1
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 2
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 2
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 3
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 3
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 4
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 4
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 5
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 5
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 6
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 6
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 7
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 7
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 8
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 8
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 9
Accuracy: 0.486, Criterion: entropy, Num_nodes

C:\Users\bersi\AppData\Local\Temp\ipykernel_8600\2830503370.py:16: RuntimeWarning: divide by zero encountered in log2
  entropy_right += -1 * (X_right_neg / X_right_count) * np.log2(X_right_neg / X_right_count)
C:\Users\bersi\AppData\Local\Temp\ipykernel_8600\2830503370.py:16: RuntimeWarning: invalid value encountered in double_scalars
  entropy_right += -1 * (X_right_neg / X_right_count) * np.log2(X_right_neg / X_right_count)
C:\Users\bersi\AppData\Local\Temp\ipykernel_8600\2830503370.py:14: RuntimeWarning: divide by zero encountered in log2
  entropy_left += -1 * (X_left_neg / X_left_count) * np.log2(X_left_neg / X_left_count)
C:\Users\bersi\AppData\Local\Temp\ipykernel_8600\2830503370.py:14: RuntimeWarning: invalid value encountered in double_scalars
  entropy_left += -1 * (X_left_neg / X_left_count) * np.log2(X_left_neg / X_left_count)


Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 10
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 11
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 11
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 12
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 12
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 13
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 13
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 14
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 14
Accuracy: 0.898, Criterion: accuracy, Num_nodes_stop: 15
Accuracy: 0.486, Criterion: entropy, Num_nodes_stop: 15
Dataset B
Accuracy: 0.79, Criterion: accuracy, Num_nodes_stop: 1
Accuracy: 0.51, Criterion: entropy, Num_nodes_stop: 1
Accuracy: 0.79, Criterion: accuracy, Num_nodes_stop: 2
Accuracy: 0.51, Criterion: entropy, Num_nodes_stop: 2
Accuracy: 0.79, Criterion: accuracy, Num_nodes_stop: 3
Accuracy: 0.51, Criterion: entropy, Num_nodes_stop: 3
Accuracy: 0.79, Criterion: accuracy, Num_n

KeyboardInterrupt: 

TextWrite cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error, for all 4 datasets (2 real world and 2 synthetic).  



In [ ]:
## Codewrite cell: Generate plots of learned decision tree classifier on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Plots only required for the accuracy criterion.
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 2 




# 4 Random Forest classifier

Write code for learning RandomForests below. Fix the following hyper parameters: (Fraction of data to learn tree=0.5, Fraction of number of features chosen in each node=0.5, num_nodes_stop=1).  Choose the number of trees to add in the forest by using a validation set. You may use a slightly modified version of the decision tree code you had written earlier.

Write code for running in the cell after the nest. (You may be asked to demonstrate your code during the viva using this cell.) 

In text cell after that report the following numbers you get by running appropriate code:

For all 4 data sets (A,B,C,D)  report the best number of trees found. Report the training and test zero-one error for those hyperparameters.

For the synthetic classification datasets (datasets A and B) in 2-dimensions, also illustrate the learned classifier for each kernel setting. Do this in the last codeWrite cell for this question.

In [ ]:
# CodeWrite cell
# Write Random Forest classifier. 

def find_accuracy(X, Y, threshold, attribute):
        X_left, X_right = X[X[:, attribute] < threshold], X[X[:, attribute] >= threshold]
        Y_left, Y_right = Y[X[:, attribute] < threshold], Y[X[:, attribute] >= threshold]

        pred_left, pred_right = np.sign(np.sum(Y_left)), np.sign(np.sum(Y_right))
        pred_left = 1 if pred_left == 0 else pred_left
        pred_right = 1 if pred_right == 0 else pred_right

        accuracy_left, accuracy_right = np.mean(pred_left == Y_left), np.mean(pred_right == Y_right)
        accuracy = (accuracy_left * X_left.shape[0] + accuracy_right * X_right.shape[0]) / X.shape[0]
        return accuracy

def find_entropy(X, Y, threshold, attribute):
    X_left, X_right = X[X[:, attribute] < threshold], X[X[:, attribute] >= threshold]
    Y_left, Y_right = Y[X[:, attribute] < threshold], Y[X[:, attribute] >= threshold]

    entropy_left, entropy_right = 0, 0

    for y in [-1, 1]:
        prob = np.sum(Y_left == y) / Y_left.shape[0]
        if(prob != 0):
            entropy_left -= prob * np.log2(prob)
        prob = np.sum(Y_right == y) / Y_right.shape[0]
        if(prob != 0):
            entropy_right -= prob * np.log2(prob)
    
    entropy = (entropy_left * X_left.shape[0] + entropy_right * X_right.shape[0]) / X.shape[0]
    return entropy

def find_best_attribute_threshold(X, Y, criterion):
    best_attribute, best_threshold = -1, -1
    best_accuracy, best_entropy = 0, np.inf

    rand_perm = np.random.permutation(X.shape[1])
    rand_perm = rand_perm[:int(0.5 * X.shape[1])]

    for attribute in rand_perm:
        for threshold in np.unique(X[:, attribute]):
            if(criterion == 'entropy'):
                entropy = find_entropy(X, Y, threshold, attribute)
                if(entropy < best_entropy):
                    best_entropy, best_attribute, best_threshold = entropy, attribute, threshold
            elif(criterion == 'accuracy'):
                accuracy = find_accuracy(X, Y, threshold, attribute)
                if(accuracy > best_accuracy):
                    best_accuracy, best_attribute, best_threshold = accuracy, attribute, threshold
    return best_attribute, best_threshold

class Node:
    def __init__(self, attribute, type, value, threshold):
        self.attribute = attribute
        self.type = type # 0: leaf, 1: internal
        self.value = value # if leaf, value is the class label, else None
        self.threshold = threshold # if internal, threshold is the threshold, else None
        self.left = None
        self.right = None

    def split_node(self, X, Y, criterion, num_nodes_stop):
        if(X.shape[0] <= num_nodes_stop):
            self.type = 0
            self.value = np.sign(np.sum(Y))
            self.value = 1 if self.value == 0 else self.value
            return self

        self.type = 1
        self.attribute, self.threshold = find_best_attribute_threshold(X, Y, criterion)
        X_left, X_right = X[X[:, self.attribute] < self.threshold], X[X[:, self.attribute] >= self.threshold]
        Y_left, Y_right = Y[X[:, self.attribute] < self.threshold], Y[X[:, self.attribute] >= self.threshold]

        node_left, node_right = Node(None, None, None, None), Node(None, None, None, None)
        self.left = node_left.split_node(X_left, Y_left, criterion, num_nodes_stop)
        self.right = node_right.split_node(X_right, Y_right, criterion, num_nodes_stop)
        return self

    def predict(self, X):
        if(self.type == 0):
            return self.value
        else:
            if(X[self.attribute] < self.threshold):
                return self.left.predict(X)
            else:
                return self.right.predict(X)

def train_decision_tree(X, Y, num_nodes_stop=1, criterion='accuracy'):
    """ 
    Returns a decision tree trained on X and Y. 
    Stops splitting nodes when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Returns a tree (In whatever format that you find appropriate)
    """
    root = Node(None, 0, None, None)
    root.split_node(X, Y, criterion, num_nodes_stop)
    return root  

def eval_decision_tree(tree, test_X):
    """ 
    Takes in a tree, and a bunch of instances X and 
    returns the tree predicted values at those instances.
    """
    return np.array([tree.predict(x) for x in test_X])

def train_random_forest(X, Y, num_trees=10, num_nodes_stop=1, criterion='accuracy', a=0.5, b=0.5):
    """ 
    Returns a random forest trained on X and Y. 
    Trains num_trees.
    Stops splitting nodes in each tree when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Fraction of data used per tree = a
    Fraction of features used in each node = b
    Returns a random forest (In whatever format that you find appropriate)
    """
    

def eval_random_forest(random_forest, test_X):
    """ 
    Takes in a  random forest object (hhowever you want to store it), and a bunch of instances X and 
    returns the tree predicted values at those instances.
    """

In [ ]:
# CodeWrite cell
# Write code for choosing the best hyperparameters (num_trees, num_nodes_stop)
# Write code here for generating the numbers that you report below.
# Repeat above for criterion set to entropy also.



TextWrite cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error, for all 4 datasets (2 real world and 2 synthetic).  


In [ ]:
## Codewrite cell: Generate plots of learned Random Forest classifier on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Plots required only for the accuracy criterion.
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 2 


# 5 AdaBoost

Write code for learning using AdaBoost below. Use 3 different weak learners below. (You may reuse code written above)

1. 1 node decision tree 
2. Decision tree of fixed depth = 3 (Root, child, grand child)
3. Decision tree of fixed depth = 7 (Root, child, grand child, ..., great^4 grand child)

Run for 50 iterations. You may use the accuracy split criterion for all the three weak learners.

Write code for running in the next cell. (You may be asked to demonstrate your code during the viva using this cell.) 

In text cell after that report the following numbers you get by running appropriate code:

For all 4 data sets (A,B,C,D)  plot the train and test accuracy vs iterations. A total of 12 plots is expected. 4 datasets * 3 weak learners. Each plot should contain two curves, train and test error.  

For the synthetic classification datasets (datasets A and B) in 2-dimensions, also illustrate the learned classifier for each weak learner setting. A total of 6 contourf style plots are expected here. Do this in the last codeWrite cell for this question.

Summarise your observations in the last textwrite cell.

In [ ]:
# Codewrite cell
# Write code to run here (no plotting)


In [ ]:
# Codewrite cell 
# Plots for iteration vs error here


In [ ]:
# Codewrite cell 
# Plots for illustrating the classifier here


Textwrite cell: